<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Silhouette Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> Silhouette function is a method of interpretation and validation of consistency within clusters of data. The function determines how well the data is clustered among clusters.The silhouette value determines the similarity of an object to its cluster (cohesion) compared to other clusters (separation). The silhouette scores are as follows:<ul style = 'font-size:16px;font-family:Arial;'>
    <li>1: Data is appropriately clustered</li>
    <li>-1: Data is not appropriately clustered</li>
    <li>0: Datum is on the border of two natural clusters</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'> 
    In this notebook we will see how we can use the Silhouette function available in Vantage.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Silhouette_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("kmeans", "computers_train1")

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
# Create teradataml DataFrame objects.
computers_train = DataFrame.from_table("computers_train1")

In [ ]:
computers_train

In [ ]:
computers_train.shape

<p style = 'font-size:16px;font-family:Arial'>As a first step we will create clusters on this data and then we will use Silhouette function to see the cohesion in the clusters.</p>

In [ ]:
#create clusters using KMeans function
KMeans_out = KMeans(id_column="id",
                     target_columns=['price', 'speed'],
                     data=computers_train,
                     num_clusters=2)
# Print the result DataFrame
KMeans_out.result

In [ ]:
#assign each id to a cluster 
KMeansPredict_out_1 = KMeansPredict(data=computers_train,
                                   object=KMeans_out,
                                   accumulate=["ram","price","speed"],
                                   output_distance=True
                                   )
      
# Print the result DataFrames.
KMeansPredict_out_1.result

<p style = 'font-size:16px;font-family:Arial'>Now let us check the Silhouette score for each function. Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(Silhouette)

In [ ]:
# Find average silhouette scores of input samples for each cluster.
Silhouette_result = Silhouette(id_column="id",
                                cluster_id_column="td_clusterid_kmeans",
                                target_columns=['"td_distance_kmeans"'],
                                data=KMeansPredict_out_1.result,
                                output_type="CLUSTER_SCORES")

# Print the result DataFrame.
Silhouette_result.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table("computers_train1")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Silhouette function reference: <a href = 'https://docs.teradata.com/search/all?query=Silhouette&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>